In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import math

In [2]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        a = math.sqrt(3) * math.sqrt(2/m.bias.data.shape[0])
        torch.nn.init._no_grad_uniform_(m.bias.data, -a, a)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        

#criterion = torch.nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
# Fix magic number

#fcn_model = torch.load('best_model')
#optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [3]:
# use_gpu = torch.cuda.is_available()
# if use_gpu:
    
#     device = torch.device("cuda:0")
#     #print(device.index)
#     #fcn_model = fcn_model.to(device)
#     fcn_model = torch.nn.DataParallel(fcn_model, output_device=device)
#     #print(fcn_model.device_ids)
    
    
def train(model, epochs, train_loader, val_loader, test_loader):
    
    use_gpu = torch.cuda.is_available()
    
    if use_gpu:
        device = torch.device("cuda:0")
        model = torch.nn.DataParallel(model)
        model.to(device)
        
        
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=5e-3)
    
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.to(device)# Move your inputs onto the gpu
                labels = Y.to(device) # Move your labels onto the gpu
            else:
                inputs, labels = X, Y # Unpack variables into inputs and labels
                
            
            outputs = model(inputs)
            loss = criterion(outputs, Variable(labels.long()))
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
                
                
            # Checking if early stop conditions met
            #if early_stop and i > early_stop_epoch:
             #   shouldStop = True
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(model, 'best_model')

        val(epoch)
        model.train()


def val(epoch):
    
    # set to evaluation mode 
    fcn_model.eval()

    softmax = nn.Softmax(dim = 1)
    
    loss = []
    pred = []
    acc = []
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(val_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
            
        if use_gpu:
            inputs = X.to(device)
            labels = Y.to(device)
            tar = tar.to(device)
        else:
            inputs, labels, tar = X, Y, tar 

        print('inputs.device', inputs.device)    

        outputs = fcn_model(inputs)
        loss.append(criterion(outputs, Variables(tar.long())))
        prediction = softmax(outputs) 
        acc.append(pixel_acc(prediction, labels))
        IOU = IOU + np.array(iou(pred, Y))
    
    acc = sum(acc)/len(acc)
    avg_loss = sum(loss)/len(loss) 
    IOU = IOU/iter  
    
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    
    return avg_loss, acc, IOU      
       
    
    
    
def test():
    
    softmax = nn.Softmax(dim = 1)
    
    pred = []
    acc = []
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(test_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
        
        if use_gpu:
            inputs = X.to(device)
            labels = Y.to(device)
            tar = tar.to(device)
        else:
            inputs, labels, tar = X, Y, tar 
                
        print('inputs.device', inputs.device)    

        outputs = fcn_model(inputs)  
        print(output.shape)
        prediction = softmax(outputs)
        acc.append(pixel_acc(prediction, labels))
        IOU = IOU + np.array(iou(prediction, Y))
        
    acc = sum(acc)/len(acc)        
    IOU = IOU/iter

    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
    return acc, IOU
    
if __name__ == "__main__":
    train_dataset = CityScapesDataset(csv_file='train.csv')
    val_dataset = CityScapesDataset(csv_file='val.csv')
    test_dataset = CityScapesDataset(csv_file='test.csv')
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=2,
                              num_workers=2,
                              shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                              batch_size=2,
                              num_workers=2,
                              shuffle=True)
    test_loader = DataLoader(dataset=test_dataset,
                              batch_size=2,
                              num_workers=2,
                              shuffle=True)
    
    
    fcn_model = FCN(n_class=34)
    fcn_model.apply(init_weights)
    #val(0)  # show the accuracy before training
    epochs     = 100
    train(fcn_model, epochs, train_loader, val_loader, test_loader)

epoch0, iter0, loss: 4.045801639556885
epoch0, iter10, loss: 3.245474100112915
epoch0, iter20, loss: 2.622002601623535
epoch0, iter30, loss: 2.063689708709717
epoch0, iter40, loss: 2.491616725921631
epoch0, iter50, loss: 2.08915638923645
epoch0, iter60, loss: 1.4533330202102661
epoch0, iter70, loss: 1.461177110671997
epoch0, iter80, loss: 1.6972006559371948
epoch0, iter90, loss: 1.729990005493164
epoch0, iter100, loss: 1.700860619544983
epoch0, iter110, loss: 2.247209072113037
epoch0, iter120, loss: 1.4781590700149536
epoch0, iter130, loss: 1.7523095607757568
epoch0, iter140, loss: 1.64076566696167
epoch0, iter150, loss: 1.7163623571395874
epoch0, iter160, loss: 2.4219937324523926
epoch0, iter170, loss: 1.8601669073104858
epoch0, iter180, loss: 1.4392993450164795
epoch0, iter190, loss: 1.6257163286209106
epoch0, iter200, loss: 1.3127657175064087
epoch0, iter210, loss: 2.081763982772827
epoch0, iter220, loss: 1.459229588508606
epoch0, iter230, loss: 1.3802525997161865
epoch0, iter240, l

KeyboardInterrupt: 